In [2]:
# Part 1 - Building the CNN
from keras.layers import Add
# Importing the Keras libraries and packages
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Conv2D,SeparableConv2D,add,GlobalAveragePooling2D, Input
from keras.layers import MaxPooling2D,Concatenate,BatchNormalization
from keras.layers import Flatten,Reshape,Dropout
from keras.layers import Dense,Input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import random
import tensorflow as tf
random.seed(5)

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
 rotation_range=20,
 width_shift_range=0.2,
 height_shift_range=0.2,
 shear_range=0.25,
 zoom_range=0.1,
 channel_shift_range = 20,
 horizontal_flip = True ,
 vertical_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [22]:
train_generator = train_datagen.flow_from_directory('C:\\Users\\Somnath\\Downloads\\datasets\\datasets\\train',
    target_size = (224, 224),
    shuffle=True,seed=42,class_mode="categorical",
    
    batch_size = 50)
test_generator = test_datagen.flow_from_directory('C:\\Users\\Somnath\\Downloads\\datasets\\datasets\\test',
    target_size = (224, 224),
    
    batch_size = 50,seed=42,class_mode="categorical",
    shuffle = False)

Found 676 images belonging to 2 classes.
Found 408 images belonging to 2 classes.


In [23]:
# Initialising the CNN
classifier = Sequential()
# Step 1 - Convolution
classifier.add(SeparableConv2D(32, (3, 3), input_shape = (224, 224, 3), activation = 'relu'))
classifier.add(BatchNormalization())
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(SeparableConv2D(64, (3, 3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(SeparableConv2D(96, (3, 3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 2, activation = 'softmax'))

In [24]:
classifier.compile(optimizer = Adam(lr=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint
#earlystopping_callback = EarlyStopping(monitor='accuracy')
checkpoint = ModelCheckpoint('output/{accuracy:.4f}.hdf5',monitor='accuracy', save_best_only=True, mode='auto')
classifier.fit_generator(train_generator,epochs =100,callbacks=[checkpoint])

Epoch 1/100
14/14 [==============================] - 22s 2s/step - loss: 0.5387 - accuracy: 0.8018
Epoch 2/100
14/14 [==============================] - 22s 2s/step - loss: 0.3223 - accuracy: 0.8964
Epoch 3/100
14/14 [==============================] - 23s 2s/step - loss: 0.2810 - accuracy: 0.9157
Epoch 4/100
14/14 [==============================] - 22s 2s/step - loss: 0.3079 - accuracy: 0.9024
Epoch 5/100
14/14 [==============================] - 23s 2s/step - loss: 0.2269 - accuracy: 0.9053
Epoch 6/100
14/14 [==============================] - 21s 2s/step - loss: 0.1621 - accuracy: 0.9364
Epoch 7/100
14/14 [==============================] - 22s 2s/step - loss: 0.1938 - accuracy: 0.9379
Epoch 8/100
14/14 [==============================] - 22s 2s/step - loss: 0.1287 - accuracy: 0.9556
Epoch 9/100
14/14 [==============================] - 22s 2s/step - loss: 0.1749 - accuracy: 0.9393
Epoch 10/100
14/14 [==============================] - 22s 2s/step - loss: 0.1485 - accuracy: 0.9482
Epoch 11/

In [26]:
from sklearn.metrics import classification_report, confusion_matrix,plot_confusion_matrix
Y_pred = classifier.predict_generator(test_generator)

In [27]:
test_generator.classes.shape

(408,)

In [28]:
y_pred = np.argmax(Y_pred, axis=1)